In [2]:
#Variables
c = 299792458
nptslambda = 1000
Noversampl = 10
noise = 0.1
ph0 = 0
FILTFACT = 10
nonlin = 0
Order = 3
fsr = 150e9
fwhml =35e-9
depth0 = 250

In [1]:
#Buttons
FILT = False
FiltType = 1
CohSum = False
AutoPhase = False
IQ = True 
BG = False
SNR = False

In [ ]:
fwhmp = nptslambda*pulsew
fwhmlw = 6.49e9            #linewidth 6.49 GHz

nfft = np.power(2,17)      #number points FFT
R = 1/10;                  #sample arm reflectivity
LB = c/2/fsr

Dlam = 3*fwhml       #sweeping range
lc = (1290e-9)       #center wavelength
le = (lc+Dlam/2)     #stopping wavelength
ls = (lc-Dlam/2)     #starting wavelength

fs = c/le                  #starting optical frequency
fs = round(fs/fsr)*fsr     #make sure it starts at intiger fsr
Df = c/ls - c/le           #sweeping bandwidth

nl = round(Df/fsr)         #number lambdas
fwhmf = c/(lc-fwhml/2) - c/(lc+fwhml/2)

In [ ]:
#BUILD FREQUENCY STEPPED VECTOR
of = np.zeros(nl)
ol = np.zeros(nl)
ok = np.zeros(nl)

for i in range(0,nl):
    of[i] = fs + i*fsr
    ol[i] = c/of[i]
    ok[i] = 1/ol[i]

#frequency vector
oof = np.zeros(nl*nptslambda)
ook = np.zeros(nl*nptslambda)

inc = 0
for i in range(0,nl):
    for j in range(0,nptslambda):
        oof[inc] = of[i]
        ook[inc] = ok[i]
        inc = inc + 1
        
npts = len(oof)

In [ ]:
#CONTINUOUS k-VECTOR
fs = of[1]-fsr
df = (of[-1]-of[1])/(npts-nptslambda+1)
fe = fs + df*npts
ofcont = np.linspace(fs,fe,npts)

In [ ]:
#GAUSSIAN FIELD ENVELOPE
ind = int(np.floor(len(ofcont)/2))
amp = np.exp(-1*np.square(ofcont-ofcont[ind])/(2*np.square(fwhmf/(2*np.sqrt(2*np.log(2))))))

In [ ]:
#PULSE TRAIN
ampp_gauss = np.exp(-1*np.square(np.linspace(1,nptslambda,nptslambda)-nptslambda/2)/(2*np.square(fwhmp/(2*np.sqrt(2*np.log(2))))))

#build pulse train
ptrain = np.zeros(nl*nptslambda)

if nptslambda  == 1:
    ampp = 1
else:
    inc = 0
    for i in range(1,nl+1):
        ptrain[inc:inc+nptslambda] = ampp_gauss 
        
        for j in range(1,nptslambda+1):
            inc = inc + 1            
    
    ampp = ptrain # Final output pulse train

# total field enveloepe
amp = amp*ampp*1/pulsew

In [ ]:
amp = np.concatenate((amp,amp,amp),axis=0)
oof = np.concatenate((oof,oof,oof),axis=0)
ofcont = np.concatenate((ofcont,ofcont,ofcont),axis=0)